In [62]:
import requests
import tempfile
import re

urls = [
    "https://www.gutenberg.org/files/67724/67724-0.txt",
    "https://www.gutenberg.org/files/67725/67725-0.txt",
    "https://www.gutenberg.org/cache/epub/21040/pg21040.txt",
    "https://www.gutenberg.org/cache/epub/55752/pg55752.txt",
    "https://www.gutenberg.org/cache/epub/54829/pg54829.txt",
    "https://www.gutenberg.org/cache/epub/55682/pg55682.txt",

    ]

text = ""

for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        with tempfile.NamedTemporaryFile(suffix=".txt") as tmp:
            with open(tmp.name, "w", encoding="utf-8") as f:
                f.write(response.text)
                text += open(tmp.name, "r").read()
        print(f"{url} downloaded successfully.")
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
    
paragraphs = text.split("\n\n")
sentences = re.split(r'(?<=[.!?])\s+', text)
len(paragraphs)

https://www.gutenberg.org/files/67724/67724-0.txt downloaded successfully.
https://www.gutenberg.org/files/67725/67725-0.txt downloaded successfully.
https://www.gutenberg.org/cache/epub/21040/pg21040.txt downloaded successfully.
https://www.gutenberg.org/cache/epub/55752/pg55752.txt downloaded successfully.
https://www.gutenberg.org/cache/epub/54829/pg54829.txt downloaded successfully.
https://www.gutenberg.org/cache/epub/55682/pg55682.txt downloaded successfully.


11288

In [63]:
for paragraph in paragraphs[50:55]:
    print(paragraph)
    print("\n")

Sobre a porta do centro desenhava-se um brasão d'armas em campo de
cinco vieiras de ouro, riscadas em cruz entre quatro rosas de prata
sobre pallas e faixas. No escudo, formado por uma brica de prata, orlada
de vermelho, via-se um elmo tambem de prata paquife de ouro e de azul, e
por timbre um meio leão de azul com uma vieira de ouro sobre a cabeça.


Um largo reposteiro de damasco vermelho, onde se reproduzia o mesmo
brasão, occultava esta porta, que raras vezes se abria, e dava para um
oratorio. Defronte, entre as duas janellas do meio, havia um pequeno
docel fechado por cortinas brancas com apanhados azues.


Cadeiras de couro de alto espaldar, uma mesa de jacarandá de pés
torneados, uma lampada de praia suspensa ao tecto, constituião a
mobilia da sala, que respirava um ar severo e triste.


Os aposentos interiores erão do mesmo gosto, menos as decorações
heraldicas; na aza do edificio, porém, esse aspecto mudava de repente, e
era substituido por um quer que seja de caprichoso e del

In [64]:
for sentence in sentences[50:55]:
    print(sentence)
    print("\n")

Tudo era grande e pomposo no scenario que a natureza, sublime artista,
tinha decorado para os dramas magestosos dos elementos, em que o homem
é apenas um simples comparsa.


No anno do graça de 1604, o lugar que acabamos de descrever estava
deserto e inculto; a cidade do Rio de Janeiro tinha-se fundado havia
menos de meio seculo, e a civilisação não tivera tempo de penetrar o
interior.


Entretanto, via-se á margem direita do rio uma casa larga e espaçosa,
construida sobre uma eminencia, e protegida de todos os lados por uma
muralha de rocha cortada a pique.


A esplanada, sobre que estava assentado o edificio, formava um
semicirculo irregular que teria quando muito cincoenta braças
quadradas: do lado do norte havia uma especie de escada de lagedo feita
metade pela natureza e metade pela arte.


Descendo dous ou tres dos largos degráos de pedra da escada,
encontrava-se uma ponte de madeira solidamente construida sobre uma
fenda larga e profunda que se abria na rocha.




In [65]:
from collections import Counter
import re

# defining a word pattern
WORD_PATTERN = r'\w+|[^\w\s]'

# tokenization
def count_words(texts: str) -> Counter:
    """counts words from texts and returns a counter object
    
    Args:
        texts (str): text for counting
    
    Returns:
        Counter(): counter with number of words 
    """
    word_count = Counter()
    for t in texts:
        word_count.update(re.findall(WORD_PATTERN, t))
    
    return word_count

word_count = count_words(paragraphs)
print(f"Word count: {len(word_count)}")
print(f"Most commom tokens: {word_count.most_common(5)}")
print(f"Least commom tokens: {word_count.most_common()[-5:]}")


Word count: 33017
Most commom tokens: [(',', 32350), ('.', 25640), ('-', 17890), ('a', 12470), ('que', 11713)]
Least commom tokens: [('abdicação', 1), ('CCI', 1), ('ganiu', 1), ('prenhe', 1), ('questões', 1)]


In [66]:
example_sentence = "As armas e os barões assinalados,"

In [67]:
def tokenize(text: str) -> list[str]: 
    """Receives a text and breaks it into words
    
    Args:
        text (str): Text to be tokenized
    
    Returns:
        list[str]: List with each tokens (words and punctuation)
    """
    return re.findall(r"[\w']+|[.,!?;]", text)

# testing
tokenize(example_sentence)

['As', 'armas', 'e', 'os', 'barões', 'assinalados', ',']

In [68]:
def generate_ngrams(n: int, tokens: list) -> list[tuple]:
    """Generates N-Grams of n size.
    
    Args:
        n (int): size of n-gram
        tokens (list): list of tokens to generate n-grams

    Returns:
        list[tuple]: list containing all n-grams in tuple format: (context, target)
    """
    n_grams = []
    context = (n-1) * ["<start>"] + tokens # add start marker to the begining of the tokens based on n-gram size

    for i in range(0, len(tokens)):
        n_grams.append((context[i:n-1+i], tokens[i]))
    
    return n_grams

# testing
example_ngrams = generate_ngrams(3, tokenize(example_sentence))
example_ngrams

[(['<start>', '<start>'], 'As'),
 (['<start>', 'As'], 'armas'),
 (['As', 'armas'], 'e'),
 (['armas', 'e'], 'os'),
 (['e', 'os'], 'barões'),
 (['os', 'barões'], 'assinalados'),
 (['barões', 'assinalados'], ',')]

In [69]:
def n_gram_counter(n_grams: list, ngram_counter: Counter, context_counter: Counter) -> tuple[Counter, Counter]:
    """Based on the extracted N-grams, count the n-gram amount and context amount
    
    Args:
        n_grams (list): list containing all n-grams in tuple format: (context, target)
    
    Returns:
        tuple[Counter, Counter]: tuple conatining the n-gram counter and context counter
    """
    for context, target in n_grams:
        n_gram = context + [target]
        ngram_counter.update([tuple(n_gram)])
        context_counter.update([tuple(context)])
    
    return ngram_counter, context_counter

# testing
n_counter, c_counter = Counter(), Counter()
n_counter, c_counter = n_gram_counter(example_ngrams, n_counter, c_counter)
c_counter, n_counter


(Counter({('<start>', '<start>'): 1,
          ('<start>', 'As'): 1,
          ('As', 'armas'): 1,
          ('armas', 'e'): 1,
          ('e', 'os'): 1,
          ('os', 'barões'): 1,
          ('barões', 'assinalados'): 1}),
 Counter({('<start>', '<start>', 'As'): 1,
          ('<start>', 'As', 'armas'): 1,
          ('As', 'armas', 'e'): 1,
          ('armas', 'e', 'os'): 1,
          ('e', 'os', 'barões'): 1,
          ('os', 'barões', 'assinalados'): 1,
          ('barões', 'assinalados', ','): 1}))

In [70]:
# test with whole corpus
ngram_counter = Counter()
context_counter = Counter()

for sentence in sentences:
    tokens = tokenize(sentence)
    n_grams = generate_ngrams(3, tokens)
    ngram_counter, context_counter = n_gram_counter(n_grams, ngram_counter, context_counter)

ngram_counter.most_common()[:3], context_counter.most_common()[:3]
    

([(('.', '.', '.'), 3678),
  (('<start>', '<start>', 'Não'), 1093),
  (('<start>', '<start>', 'O'), 907)],
 [(('<start>', '<start>'), 22954), ((',', 'e'), 3806), (('.', '.'), 3787)])

In [71]:
candidates = [
    (ngram[-1], count)
    for ngram, count in ngram_counter.items()
]

candidates

[('START', 5),
 ('OF', 1),
 ('THE', 6),
 ('PROJECT', 12),
 ('GUTENBERG', 12),
 ('EBOOK', 12),
 ('67724', 2),
 ('J', 1),
 ('.', 1),
 ('DE', 3),
 ('ALENCAR', 2),
 ('O', 2),
 ('GUARANY', 2),
 ('ROMANCE', 2),
 ('BRAZILEIRO', 2),
 ('QUINTA', 2),
 ('EDIÇÃO', 2),
 ('TOMO', 2),
 ('PRIMEIRO', 1),
 ('RIO', 1),
 ('DE', 1),
 ('JANEIRO', 5),
 ('B', 3),
 ('.', 3),
 ('L', 2),
 ('.', 2),
 ('GARNIER', 4),
 (',', 4),
 ('LIVREIRO', 4),
 ('EDITOR', 4),
 ('71', 3),
 (',', 3),
 ('RUA', 2),
 ('DO', 2),
 ('OUVIDOR', 2),
 (',', 2),
 ('71', 2),
 ('PARIS', 2),
 ('.', 2),
 ('E', 2),
 ('.', 86),
 ('MELLIER', 2),
 (',', 2),
 ('17', 2),
 (',', 2),
 ('RUA', 2),
 ('SÉGUIER', 2),
 ('.', 2),
 ('Ficão', 2),
 ('reservados', 2),
 ('os', 2),
 ('direitos', 2),
 ('de', 3),
 ('propriedade', 2),
 ('.', 3),
 ('1883', 2),
 ('AO', 1),
 ('LEITOR', 1),
 ('Publicando', 1),
 ('este', 1),
 ('livro', 1),
 ('em', 1),
 ('1857', 1),
 (',', 2),
 ('se', 1),
 ('disse', 2),
 ('ser', 1),
 ('aquella', 1),
 ('primeira', 1),
 ('edição', 1),
 ('uma

In [91]:
import random

def generate_text(n: int, ngram_counter: Counter, context_counter: Counter, max_words: int, initial_text: str): 

    """Generates text using the n_gram and context counter ("trained model")
    
    Args:
        n (int): size of n-gram
        ngram_counter (Counter): Counter object containing the count of ngrams
        context_counter (Counter): Counter object containing the count of context
        initial_text (str): initial text to begin the generation
        max_words (int): max number of generated words

    Returns:
        stream of strings form generated text
    """
    initial_text = tokenize(initial_text)

    output = initial_text.copy()

    if len(initial_text) >= n-1:
        context = initial_text[-(n - 1):]
    else:
        context = ["<start>"] * (n - 1 - len(initial_text)) + initial_text
    
    for _ in range(max_words):
        candidates = [
            (ngram[-1], count)
            for ngram, count in ngram_counter.items()
            if ngram[:-1]==tuple(context)
        ]

        if not candidates:
            break

        words, counts = zip(*candidates)
        total = sum(counts)
        probabilities = [count/total for count in counts]

        next_word = random.choices(words, weights=probabilities, k=1)[0]

        output.append(next_word)
    
    return " ".join(output)

# testing
generate_text(
    3,
    ngram_counter,
    context_counter,
    20,
    "se"
)


'se ella nos tinha ella atino elle ella elle elle atino visseis atino atino nos visseis nos tinha tinha nos tinha'